In [1]:
import numpy as np
from openmm.app import *
from openmm.unit import *
from openmm import *
from openmm.app.modeller import Modeller
from pdbfixer import PDBFixer

In [2]:
raw = PDBFixer('L_allo_ile.pdb')
raw.findMissingResidues()
raw.findMissingAtoms()
raw.addMissingAtoms()

In [3]:
forcefield = ForceField('amber14/protein.ff14SB.xml', 'amber14/tip3p.xml')
mdl = Modeller(raw.topology, raw.positions)
mdl.addHydrogens(forcefield, pH=7.0)
mdl.addSolvent(forcefield, 'tip3p', padding=1*nanometers)

PDBFile.writeFile(mdl.topology, mdl.positions, open('L_allo_solvated_tmp.pdb', 'w'))

In [4]:
system = forcefield.createSystem(mdl.topology,
                                 nonbondedMethod=PME,
                                 nonbondedCutoff=1*nanometer)

integrator = LangevinMiddleIntegrator(293*kelvin,
                                      1.0/picoseconds,
                                      2.0*femtoseconds)

platform = openmm.Platform.getPlatformByName('CUDA')
platform_properties = {'DeviceIndex':'0'}
simulation = Simulation(mdl.topology, system, integrator, platform=platform,
                        platformProperties=platform_properties)
simulation.context.setPositions(mdl.positions)
simulation.minimizeEnergy()

# Trajectory saved every 2 ps
simulation.reporters.append(PDBReporter('L_allo_NVT_equilibration.pdb', 1000))
simulation.reporters.append(CheckpointReporter('L_allo_NVT_equilibrated.chk', reportInterval=1000))
simulation.reporters.append(StateDataReporter('L_allo_NVT_file.txt', 1000, step=True, potentialEnergy=True, temperature=True))

# NVT equilibration for 100 ps 
simulation.step(50000)